In [2]:
import os
os.environ['HF_HOME'] = '/home/bachirijihane/LLM RAG/models'

from langchain import OpenAI, FAISS, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings

import tqdm
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [4]:
# pip install faiss-gpu

In [5]:
embeddings = HuggingFaceEmbeddings()

In [6]:
# Prepare vector store (FAISS) with IPPC report
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)
loader = TextLoader("DonneesElec.txt")
db = FAISS.from_documents(loader.load_and_split(text_splitter), HuggingFaceEmbeddings())

In [7]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             quantization_config=bnb_config,
                                             revision="main")

KeyError: 'mistral'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [ ]:
# Prepare QA chain
PROMPT_TEMPLATE = """Vous êtes Jihane, une ingénieure électricienne et professeure qui répond à des questions sur le génie électrique uniquement.
Vous recevrez une question et des extraits pertinents d'une leçon en français et vous ne pouvez répondre qu'à des questions relatives au génie électrique.
Veuillez fournir des réponses courtes et claires sur le contexte fourni et uniquement en français. Soyez poli et serviable.

Context:
{context}

Question:
{question}

Your answer:
"""

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0,
    device_map="auto",
    batch_size=5,
    pad_token_id=tokenizer.eos_token_id,
    return_full_text = False)
pipe.tokenizer.pad_token_id = model.config.eos_token_id

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
llm_chain.invoke({"context":"", # contexte forcé a vide 
                  "question": "Qu'est ce que le flux conventionnel?"})

In [ ]:
query = "Qu'est ce que le flux conventionnel ?"
docs = db.similarity_search(query)
for doc in docs:
    print(doc.page_content)

In [ ]:
query = "Qu'est ce que le flux conventionnel ?" 

retriever = db.as_retriever()

rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

rag_chain.invoke(query)

In [ ]:
var = rag_chain.invoke(query)

In [ ]:
var["text"]